# Processing cyclic voltammograms from multiple sets of files

## LIbrary imports and some default values for plotting

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from ipywidgets import widgets
from ipywidgets import Button, HBox, VBox, Text, interactive_output

In [ ]:
ref_electrodes = ['Li/Li$^+$', 'Na/Na$^+$', 'Hg/HgSO$_4$', 'Ag/AgCl', 'Carbon', 'Ag']
normalization_options = ['Current density (A/g)', 'Specific Capacitance (F/g)', 'Specific Capacity (mAh/g)', 
                         'Specific Charge (C/g)', 'Volumetric Capacitance (F/cm$^3$)', 'Volumetric Capacity (mAh/cm$^3$)',
                        'Volumetric Charge (C/cm$^3$)', 'Areal Capacitance (F/cm$^2$)', 'Areal Capacity (mAh/cm$^2$)',
                        'Areal Charge (C/cm$^2$)']

## Reading in files, preprocessing data, and input of user constants

In [ ]:
# Enter path to folder containing your data in quotes.
rootdir = ''

In [ ]:
# Enter your scan rates as a comma separated list in the square brackets, e.g., [5, 10, 20, 50, 100].
scan_rates = []

In [ ]:
# Replace 'None' with your values. Okay to leave as 'None' if you don't have a value.
electrode_mass = None #in mg
electrode_area = None #in cm^2
electrode_volume = None #in cm^3

In [ ]:
def create_file_dict(rootdir, scan_rates):
    '''
    Creates a dictionary of file paths that correspond to the cell 
    data that will be processed. Files must be named such that they 
    are pre-sorted. Files are grouped by scan rate.
    '''

    file_dict = {}

    directory_path = [x[0] for x in os.walk(rootdir)][1:]
    file_path = [x[2] for x in os.walk(rootdir)][1:]

    full_path_list = []
    for idx, nested_list in enumerate(file_path):
        temp = []
        for element in nested_list:
            temp.append('{}\\{}'.format(directory_path[idx], element))
        full_path_list.append(temp)

    for scan_rate, path in zip(scan_rates, full_path_list):
        file_dict[scan_rate] = path

    return file_dict


In [ ]:
def cvs_avg_and_variance(data):
    '''
    Calculates average potential, averager current, and current variance 
    for the second to n - 1 cycles for a given scan rate from a BioLogic 
    txt file.
    '''

    cycle_num = data['cycle number'].unique()

    current = []
    for n in cycle_num[1:-1]:
        current.append(np.asarray(
            data['<I>/mA'][data['cycle number'] == n]))
    current = pd.DataFrame(current).transpose()
    avg_current = np.mean(current, axis=1)
    current_var = np.var(current, axis=1)

    potential = []
    for n in cycle_num[1:-1]:
        potential.append(np.asarray(
            data['Ewe/V'][data['cycle number'] == n]))
    potential = pd.DataFrame(potential).transpose()
    avg_potential = potential.mean(axis=1)

    return list(avg_potential), list(avg_current), list(current_var)


In [ ]:
def create_and_sort_data_dict(file_dict):
    '''
    Creates and then sorts a dictionary of the number of cycles,
    average potentials, average currents, and current variances 
    from the files listed in the file_dict. Entries are grouped
    by scan rate.
    '''

    data_dict = {}

    for key in file_dict:
        data_dict[key] = {'cycles': [],
                          'avg_potential': [],
                          'avg_current': [],
                          'current_var': []
                          }

    for key in file_dict:

        for element in file_dict[key]:

            with open(element, 'r') as f:
                data = pd.read_table(f)
                cycles = max(data['cycle number']) - 2
                avg_potential, avg_current, current_var = cvs_avg_and_variance(
                    data)

            data_dict[key]['cycles'].append(cycles)
            data_dict[key]['avg_potential'].append(avg_potential)
            data_dict[key]['avg_current'].append(avg_current)
            data_dict[key]['current_var'].append(current_var)

    for key in data_dict:

        for i in range(len(data_dict[key]['avg_potential'])):

            for idx, val in enumerate(data_dict[key]['avg_potential'][i]):

                if data_dict[key]['avg_potential'][i][0] != max(data_dict[key]['avg_potential'][i]):

                    pop_potential = data_dict[key]['avg_potential'][i].pop(
                        0)
                    data_dict[key]['avg_potential'][i].append(
                        pop_potential)

                    pop_current = data_dict[key]['avg_current'][i].pop(0)
                    data_dict[key]['avg_current'][i].append(pop_current)

                    pop_current_var = data_dict[key]['current_var'][i].pop(
                        0)
                    data_dict[key]['current_var'][i].append(
                        pop_current_var)

    return data_dict


In [ ]:
def check_and_downsample(data_dict):
    '''
    Checks number of data points for each entry in the data_dict. 
    Downsamples any entry that is longer than the shortet entry. 
    '''

    lengths = []

    for key in data_dict:
        for idx, val in enumerate(data_dict[key]['avg_potential']):
            lengths.append(len(val))

    for key in data_dict:
        for idx, val in enumerate(data_dict[key]['avg_potential']):
            if len(val) != min(lengths):
                points_to_drop = np.round(np.linspace(0, len(data_dict[key]['avg_potential'][idx]) - 1,
                                                      len(data_dict[key]['avg_potential'][idx]) -
                                                      min(lengths))).astype(int)

                potential_update = [
                    data_dict[key]['avg_potential'][idx][i] for i in points_to_drop]
                current_update = [data_dict[key]['avg_current'][idx][i]
                                  for i in points_to_drop]
                var_update = [data_dict[key]['current_var'][idx][i]
                              for i in points_to_drop]

                potential_dropped = [item for item in data_dict[key]
                                     ['avg_potential'][idx] if item not in potential_update]
                current_dropped = [item for item in data_dict[key]
                                   ['avg_current'][idx] if item not in current_update]
                var_dropped = [item for item in data_dict[key]
                               ['current_var'][idx] if item not in var_update]

                data_dict[key]['avg_potential'][idx] = potential_dropped
                data_dict[key]['avg_current'][idx] = current_dropped
                data_dict[key]['current_var'][idx] = var_dropped
                

In [ ]:
def get_weighted_avgs_std(data_dict):
    '''
    Generates dictionary containing the weighted averages of the
    potenials and currents and the current standard deviation for
    each scan rate in the data_dict. Weighted averages use the
    number of cycles recorded at each scan rate
    '''

    final_data = {}

    for key in data_dict:

        weighted_avg_potential = np.zeros(
            len(data_dict[key]['avg_potential'][0])
        )
        weighted_avg_current = np.zeros(
            len(data_dict[key]['avg_current'][0])
        )
        avg_current_std = np.zeros(
            len(data_dict[key]['current_var'][0])
        )

        for idx, val in enumerate(data_dict[key]['avg_potential']):
            weighted_avg_potential += (np.asarray(val) * (
                data_dict[key]['cycles'][idx] / sum(data_dict[key]['cycles'])))

        for idx, val in enumerate(data_dict[key]['avg_current']):
            weighted_avg_current += (np.asarray(val) * (
                data_dict[key]['cycles'][idx] / sum(data_dict[key]['cycles'])))

        for idx, val in enumerate(data_dict[key]['current_var']):
            avg_current_std += np.asarray(val)

        final_data[key] = {
            'w_avg_potential': weighted_avg_potential,
            'w_avg_current': weighted_avg_current,
            'current_std_dev': np.sqrt(avg_current_std)
        }

    return final_data


In [ ]:
file_dict = create_file_dict(rootdir=rootdir, scan_rates=scan_rates)
data_dict = create_and_sort_data_dict(file_dict=file_dict)
check_and_downsample(data_dict=data_dict)
final_data = get_weighted_avgs_std(data_dict=data_dict)

## Plotting BioLogic cyclic voltammograms

In [ ]:
def plot_CVs_biologic(ref, yaxis, Error):
    '''
    Plots average voltammograms from file list with error bands.
    '''
    global fig
    
    fig, ax = plt.subplots(figsize=(6, 6), dpi=150)

    for key, rate in zip(final_data, scan_rates):
        
        potential = final_data[key]['w_avg_potential']
        current = final_data[key]['w_avg_current']
        std = final_data[key]['current_std_dev']
        
        if 'Capacity' in yaxis:
            capacity = (current * (max(potential) - min(potential))) / (3.6 * rate)
            capacity_err = (std * (max(potential) - min(potential))) / (3.6 * rate)

            if 'Specific' in yaxis:
                ax.plot(potential, 
                        capacity * 1000 / electrode_mass, 
                        linewidth=2, label=(f"{rate} mV/s"))
                if Error:
                    ax.fill_between(potential, 
                                    (capacity + capacity_err) * 1000 / electrode_mass, 
                                    (capacity - capacity_err) * 1000 / electrode_mass, 
                                    alpha=0.4)
            elif 'Volumetric' in yaxis:
                ax.plot(potential, 
                        capacity / electrode_volume, 
                        linewidth=2, label=(f"{rate} mV/s"))
                if Error:
                    ax.fill_between(potential, 
                                    (capacity + capacity_err) / electrode_volume, 
                                    (capacity - capacity_err) / electrode_volume, 
                                    alpha=0.4)
            elif 'Areal' in yaxis:
                ax.plot(potential, 
                        capacity / electrode_area, 
                        linewidth=2, label=(f"{rate} mV/s"))
                if Error:  
                    ax.fill_between(potential, 
                                    (capacity + capacity_err) / electrode_area, 
                                    (capacity - capacity_err) / electrode_area, 
                                    alpha=0.4)

        elif 'Capacitance' in yaxis:
            capacitance = current / rate
            capacitance_err = std / rate

            if 'Specific' in yaxis:
                ax.plot(potential, 
                        capacitance * 1000 / electrode_mass, 
                        linewidth=2, label=(f"{rate} mV/s"))
                if Error:
                    ax.fill_between(potential, 
                                    (capacitance + capacitance_err) * 1000 / electrode_mass, 
                                    (capacitance - capacitance_err) * 1000 / electrode_mass, 
                                    alpha=0.4)
            elif 'Volumetric' in yaxis:
                ax.plot(potential, 
                        capacitance / electrode_volume, 
                        linewidth=2, label=(f"{rate} mV/s"))
                if Error:
                    ax.fill_between(potential, 
                                    (capacitance + capacitance_err) / electrode_volume, 
                                    (capacitance - capacitance_err) / electrode_volume, 
                                    alpha=0.4)
            elif 'Areal' in yaxis:
                ax.plot(potential, 
                        capacitance / electrode_area, 
                        linewidth=2, label=(f"{rate} mV/s"))
                if Error:  
                    ax.fill_between(potential, 
                                    (capacitance + capacitance_err) / electrode_area, 
                                    (capacitance - capacitance_err) / electrode_area, 
                                    alpha=0.4)

        elif 'Charge' in yaxis:
            charge = (current * (max(potential) - min(potential))) / rate
            charge_err = (std * (max(potential) - min(potential))) / rate

            if 'Specific' in yaxis:
                ax.plot(potential, charge * 1000 / electrode_mass, 
                        linewidth=2, label=(f"{rate} mV/s"))
                if Error:
                    ax.fill_between(potential, 
                                    (charge + charge_err) * 1000 / electrode_mass, 
                                    (charge - charge_err)* 1000 / electrode_mass, 
                                    alpha=0.4)
            elif 'Volumetric' in yaxis:
                ax.plot(potential, charge / electrode_volume, 
                        linewidth=2, label=(f"{rate} mV/s"))
                if Error:
                    ax.fill_between(potential, 
                                    (charge + charge_err) / electrode_volume, 
                                    (charge - charge_err) / electrode_volume, 
                                    alpha=0.4)
            elif 'Areal' in yaxis:
                ax.plot(potential, charge / electrode_area, 
                        linewidth=2, label=(f"{rate} mV/s"))
                if Error:  
                    ax.fill_between(potential, 
                                    (charge + charge_err) / (electrode_area), 
                                    (charge - charge_err) / (electrode_area), 
                                    alpha=0.4)

        else:
            ax.plot(potential, current / electrode_mass, linewidth=2, label=(f"{rate} mV/s"))
            if Error:
                ax.fill_between(potential, (current + std) / electrode_mass, 
                                (current - std) / electrode_mass, 
                                alpha=0.4)
                          
    
    ax.tick_params(which='both', labelsize=14, width=2, length=5)
    ax.legend(frameon=False)

    ax.set_xlabel(f'Potential vs. {ref}', fontsize=18)
    ax.set_ylabel(f'{yaxis}', fontsize=18)

In [ ]:
def make_figure():
    '''
    Generates interactive figure using the plot_CVs_biologic function.
    '''
    refs = widgets.Dropdown(options=ref_electrodes)
    yaxis_choice = widgets.Dropdown(options=normalization_options)
    err = widgets.Checkbox(description='Error', indent=False)
    def on_button_clicked(b):
        fig.savefig("test.png")

    button = Button(description="Save Figure")
    button.on_click(on_button_clicked)

    ui = HBox([refs, yaxis_choice, err, button])

    out = widgets.interactive_output(plot_CVs_biologic, {'ref': refs, 'yaxis': yaxis_choice, 'Error': err})
    display(ui, out)

In [ ]:
make_figure()